In [1]:
import pandas as pd
import numpy as np

In [2]:
cd Documents/Budget-Pages

/Users/jdw686/Documents/Budget-Pages


### 1. Preparing the data manipulation for the department pages for the City's General Fund

In [3]:
budget_pages = pd.read_csv('department pages.csv')

In [4]:
budget_pages.columns

Index(['a_account_type', 'a_fund_seg1', 'master_object', 'master_project',
       'master_org', 'account_name', 'department', 'division', 'cy_revised',
       'ly_actual', 'ly_actual_closed', 'ly2_actual', 'object_name',
       'character_code', 'character_code_name', 'org_name', 'default_year',
       'default_period', 'munis_year', 'munis_period', 'fund_name',
       'dept_name', 'division_name', 'budget_projection_used', 'budget_org',
       'budget_object', 'budget_project', 'ny_budget_prelim',
       'ny_budget_adopted', 'ny2_budget_prelim', 'ny2_budget_adopted'],
      dtype='object')

In [5]:
#Choose the columns needed
budget_pages = budget_pages[['a_account_type', 'a_fund_seg1', 'master_object', 'master_project',
       'master_org', 'account_name', 'department', 'division', 'cy_revised',
       'ly_actual', 'ly_actual_closed', 'ly2_actual',
       'character_code', 'character_code_name', 'org_name', 'munis_year', 'fund_name',
       'dept_name', 'division_name',
       'ny_budget_adopted', 'ny2_budget_adopted']]

In [7]:
#Rename the columns, the names above are the default names in the sql database of the City's financial system
budget_pages.columns = ['Account Type', 'Fund Number', 'Account Number', 'Project Number',
       'Organization Number', 'Account Name', 'Department Number', 'Division Number', 
                        '2018 Revised Budget',
      '2017 Actual', '2017 Actual Closed', 
                        '2016 Actual',
       'Character Code', 'Character Code Name', 'Organization Name', 'munis_year', 'Fund Name',
       'Department Name', 'Division Name',
       '2019 Adopted Budget', '2020 Adopted Budget']


In [8]:
#Rearrange the columns. It is important to have the budgeted and actual numbers on the right side
#But, this can and should be done through the SQL Query
budget_pages = budget_pages[['Account Type', 'Fund Number', 'Fund Name', 'Department Number', 'Department Name', 
                            'Division Number', 'Division Name', 'Character Code', 'Character Code Name', 
                             'Organization Number', 'Organization Name', 
                            'Account Number', 'Account Name', '2016 Actual', '2017 Actual', '2018 Revised Budget', 
                            '2019 Adopted Budget', '2020 Adopted Budget']]

In [28]:
budget_pages.head(7)

,Account Type,Fund Number,Fund Name,Department Number,Department Name,Division Number,Division Name,Character Code,Character Code Name,Organization Number,Organization Name,Account Number,Account Name,2016 Actual,2017 Actual,2018 Revised Budget,2019 Adopted Budget,2020 Adopted Budget
0,E,1,GENERAL,1,CITY COUNCIL,1,City Council,10,SALARIES & WAGES,151160,City Council,110000,Salaries and Wages,52200.00,52200.00,52200,52200,52200
1,E,1,GENERAL,3,CITY MANAGER,2,City Manager,10,SALARIES & WAGES,251310,City Manager,110000,Salaries and Wages,353904.70,375601.78,424700,251200,253600
2,E,1,GENERAL,5,ADMINISTRATIVE SERVICES,3,Communications,10,SALARIES & WAGES,351310,Communications,110000,Salaries and Wages,0.00,0.00,0,113100,121300
3,E,1,GENERAL,3,CITY MANAGER,4,Economic Development,10,SALARIES & WAGES,455870,Economic Development,110000,Salaries and Wages,308750.19,175415.09,234500,244800,252100
4,E,1,GENERAL,5,ADMINISTRATIVE SERVICES,5,City Clerk,10,SALARIES & WAGES,551420,City Clerk,110000,Salaries and Wages,225348.34,259289.96,345600,360800,374500
5,E,1,GENERAL,5,ADMINISTRATIVE SERVICES,6,Human Resources,10,SALARIES & WAGES,651810,Human Resources,110000,Salaries and Wages,119259.00,133185.61,204500,230400,240500
6,E,1,GENERAL,4,FINANCE,7,Finance,10,SALARIES & WAGES,751420,Finance,110000,Salaries and Wages,447993.44,504496.25,716700,602000,602000


In [ ]:
# budget_page_summary will be used in a further update to replace all of the individual accounts in salaries and personnel benefits with a summary line

# budget_page_summary = budget_pages[(budget_pages['Department Number'] == 5) & (budget_pages['Division Number'] == 8) & (budget_pages['Account Type'] == 'E')].groupby(
#     ['Character Code', 'Character Code Name']).sum().iloc[:, -5:]
# total_summary = budget_page_summary.apply(np.sum)
# total_summary['Character Code'] = ''
# total_summary['Character Code Name'] = 'TOTAL'
# budget_page_summary = budget_page_summary.reset_index().append(pd.DataFrame(total_summary.values, index=list(total_summary.keys())).T).fillna('').set_index(
#     ['Character Code', 'Character Code Name'])

# budget_page_summary.reset_index()

In [9]:
budget_page_detail = budget_pages[(budget_pages['Department Number'] == 5) & 
                                  (budget_pages['Division Number'] == 8) & 
                                  (budget_pages['Account Type'] == 'E')].groupby(
    ['Organization Number', 'Account Number', 'Account Name']).sum().iloc[:, -5:].reset_index().set_index('Organization Number', 'Account Number')
total_detail = budget_page_detail.apply(np.sum)
total_detail['Account Number'] = ''
total_detail['Account Name'] = 'Total'
budget_page_detail = budget_page_detail.reset_index().append(pd.DataFrame(total_detail.values, index=list(total_detail.keys())).T).fillna('').set_index(
    ['Organization Number', 'Account Number', 'Account Name'])

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [10]:
budget_page_detail.reset_index()

,Organization Number,Account Number,Account Name,2016 Actual,2017 Actual,2018 Revised Budget,2019 Adopted Budget,2020 Adopted Budget
0,851880,110000,Salaries and Wages,191724.17,224145.58,310300,336400,358900
1,851880,120000,Overtime,1262.13,561.72,0,0,0
2,851880,210000,Medicare,2799.02,3259.94,4500,4880,5200
3,851880,220000,Retirement,17887.61,26863.42,39405,43160,46040
4,851880,230000,Medical Insurance,20770.27,23537.57,46905,48585,50350
5,851880,231000,HSA Employee Benefit,0.00,1437.50,1500,1500,1500
6,851880,240000,Worker's Compensation,531.77,597.16,880,960,960
7,851880,250000,Unemployment,578.90,449.16,620,675,720
8,851880,270000,Social Security,11965.06,13628.67,19240,0,0
9,851880,271000,401a-Social Sec Replacement,0.00,0.00,0,20860,22250


In [11]:
#this corresponds to the separate table in Adopted Budget document, detailing spending on professional contract services
#this may be added in the function at a future update
budget_page_prof_srvs = budget_pages[(budget_pages['Department Number'] == 5) & 
                                  (budget_pages['Division Number'] == 8) & 
                                  (budget_pages['Account Type'] == 'E') &
                                 (budget_pages['Account Number'] >= 410000) &
                                 (budget_pages['Account Number'] < 420000)].groupby(
    ['Organization Number', 'Account Number', 'Account Name']).sum().iloc[:, -5:]

In [12]:
total = budget_page_prof_srvs.apply(np.sum)
total['Account Name'] = 'Total'
budget_pages_prof_srvs = budget_page_prof_srvs.reset_index().append(pd.DataFrame(total.values, index=list(total.keys())).T).fillna('').set_index(
    ['Organization Number', 'Account Number', 'Account Name'])

In [14]:
budget_pages_prof_srvs.reset_index()

,Organization Number,Account Number,Account Name,2016 Actual,2017 Actual,2018 Revised Budget,2019 Adopted Budget,2020 Adopted Budget
0,851880,410000,Professional Services,34004.93,3696.25,35000.0,24000.0,24000.0
1,851880,410400,Website,10443.01,18665.11,10000.0,10000.0,10000.0
2,851880,410700,Online Video Streaming,5018.78,3495.00,3000.0,4000.0,4000.0
3,851880,410750,Channel 21 Video Production,14496.08,19431.91,30000.0,45000.0,45000.0
4,851880,414100,Software Licensing Fees,68004.79,100859.59,139300.0,139440.0,213740.0
5,,,Total,131967.59,146147.86,217300.0,222440.0,296740.0


### 2. Create the function

- this simply combines the budget_page_detail code into a function, where the parameters are defined in the function instead of directly in the code

In [15]:
def department_page(department, division, account_type = 'E'):
    budget_page_detail = budget_pages[(budget_pages['Department Number'] == department) & 
                                      (budget_pages['Division Number'] == division) & 
                                      (budget_pages['Account Type'] == account_type)]
    budget_page_detail = budget_page_detail.groupby(['Organization Number', 'Account Number', 'Account Name'])
    budget_page_detail = budget_page_detail.sum().iloc[:, -5:]
    budget_page_detail = budget_page_detail.reset_index().set_index('Organization Number', 'Account Number')
    total_row = budget_page_detail.apply(np.sum)
    total_row['Account Number'] = ''
    total_row['Account Name'] = 'Total'
    total_row = pd.DataFrame(total_row.values, index=list(total_row.keys())).T
    budget_page_detail = budget_page_detail.reset_index().append(total_row).fillna('').set_index(
        ['Organization Number', 'Account Number', 'Account Name'])
    budget_page_detail = budget_page_detail.reset_index()
    return budget_page_detail

In [26]:
test_page = department_page(4, 7)
test_page

,Organization Number,Account Number,Account Name,2016 Actual,2017 Actual,2018 Revised Budget,2019 Adopted Budget,2020 Adopted Budget
0,751420,110000,Salaries and Wages,447993.44,504496.25,716700,602000,602000
1,751420,120000,Overtime,0.00,604.18,0,0,0
2,751420,210000,Medicare,6402.15,7207.62,10390,8730,8725
3,751420,220000,Retirement,49756.80,59274.88,88665,77235,77235
4,751420,230000,Medical Insurance,61432.07,71341.13,131395,100590,104015
5,751420,231000,HSA Employee Benefit,1050.00,0.00,1500,0,0
6,751420,240000,Worker's Compensation,979.75,1106.99,1825,1515,1395
7,751420,250000,Unemployment,1344.02,1010.22,1435,1205,1200
8,751420,270000,Social Security,27775.88,30610.56,44440,2065,0
9,751420,271000,401a-Social Sec Replacement,0.00,0.00,0,35260,37325


0.5536837283047289